In [1]:
import numpy as np
import pandas as pd
import psycopg2
import os

In [2]:
#Random Selection. Just thought experiment 1 looked best, because it is the most normal
#Conditions 
#material = wax
#feedrate = 6
#clamp_pressure = 4
#tool_condition = unworn
#machining_finalized = yes
#passed_visual_inspection = yes

cwd = os.getcwd()
path1 = os.path.join(cwd,r'experiment_01.csv')

experiment1 = pd.read_csv(path1)
experiment1 = experiment1.iloc[:,:-1]
print(experiment1)
experiment1.dtypes

      X1_ActualPosition  X1_ActualVelocity  X1_ActualAcceleration  \
0                 198.0              0.000                   0.00   
1                 198.0            -10.800                -350.00   
2                 196.0            -17.800                  -6.25   
3                 194.0            -18.000                   0.00   
4                 193.0            -17.900                 -18.80   
...                 ...                ...                    ...   
1050              141.0              0.175                  87.50   
1051              141.0             -0.150                 -87.50   
1052              141.0             -0.175                 -87.50   
1053              141.0              0.150                  87.50   
1054              141.0              0.175                  93.70   

      X1_CommandPosition  X1_CommandVelocity  X1_CommandAcceleration  \
0                  198.0                 0.0                0.000000   
1                  198.0   

X1_ActualPosition            float64
X1_ActualVelocity            float64
X1_ActualAcceleration        float64
X1_CommandPosition           float64
X1_CommandVelocity           float64
X1_CommandAcceleration       float64
X1_CurrentFeedback           float64
X1_DCBusVoltage              float64
X1_OutputCurrent             float64
X1_OutputVoltage             float64
X1_OutputPower               float64
Y1_ActualPosition            float64
Y1_ActualVelocity            float64
Y1_ActualAcceleration        float64
Y1_CommandPosition           float64
Y1_CommandVelocity           float64
Y1_CommandAcceleration       float64
Y1_CurrentFeedback           float64
Y1_DCBusVoltage              float64
Y1_OutputCurrent             float64
Y1_OutputVoltage             float64
Y1_OutputPower               float64
Z1_ActualPosition            float64
Z1_ActualVelocity            float64
Z1_ActualAcceleration        float64
Z1_CommandPosition           float64
Z1_CommandVelocity           float64
Z

In [3]:
    import psycopg2
    from io import StringIO

    # Connection parameters
    param_dic = {
        "host"      : "localhost",
        "database"  : "sensordata",
        "user"      : "postgres",
        "password"  : "JNt8hSW4l+MJ]p;.g[WG"
    }

    #Credit to Naysan Saran. https://naysan.ca/2020/06/21/pandas-to-postgresql-using-psycopg2-copy_from/

    def connect(params_dic):
        """ Connect to the PostgreSQL database server """
        conn = None
        try:
            # connect to the PostgreSQL server
            print('Connecting to the PostgreSQL database...')
            conn = psycopg2.connect(**params_dic)
        except (Exception, psycopg2.DatabaseError) as error:
            print(error)
            sys.exit(1) 
        print("Connection successful")
        return conn


    def copy_from_stringio(conn, df, table):
        """
        Here we are going save the dataframe in memory 
        and use copy_from() to copy it to the table
        """
        # save dataframe to an in memory buffer
        buffer = StringIO()
        df.to_csv(buffer, index_label='id', header=False)
        buffer.seek(0)

        cursor = conn.cursor()
        try:
            cursor.copy_from(buffer, table, sep=",")
            conn.commit()
        except (Exception, psycopg2.DatabaseError) as error:

            print("Error: %s" % error)
            conn.rollback()
            cursor.close()
            return 1
        print("copy_from_stringio() done")
        cursor.close()

In [4]:
conn = connect(param_dic) # connect to the database
cursor = conn.cursor() # set cursor

# CREATE TABLE
list1 = list(experiment1.keys()) #use column labels for SQL query
string1 = "CREATE TABLE CNC_mill_tool_wear (id int"
for label in list1:
    string1 += (", " + label + " decimal")
string1 = string1 + ");" #Last column is String and needs to be handled seperately
print(string1)
cursor.execute(string1) #execute SQL query

copy_from_stringio(conn, experiment1, "CNC_mill_tool_wear") # copy the dataframe to SQL
conn.close() # close the connection

Connecting to the PostgreSQL database...
Connection successful
CREATE TABLE CNC_mill_tool_wear (id int, X1_ActualPosition decimal, X1_ActualVelocity decimal, X1_ActualAcceleration decimal, X1_CommandPosition decimal, X1_CommandVelocity decimal, X1_CommandAcceleration decimal, X1_CurrentFeedback decimal, X1_DCBusVoltage decimal, X1_OutputCurrent decimal, X1_OutputVoltage decimal, X1_OutputPower decimal, Y1_ActualPosition decimal, Y1_ActualVelocity decimal, Y1_ActualAcceleration decimal, Y1_CommandPosition decimal, Y1_CommandVelocity decimal, Y1_CommandAcceleration decimal, Y1_CurrentFeedback decimal, Y1_DCBusVoltage decimal, Y1_OutputCurrent decimal, Y1_OutputVoltage decimal, Y1_OutputPower decimal, Z1_ActualPosition decimal, Z1_ActualVelocity decimal, Z1_ActualAcceleration decimal, Z1_CommandPosition decimal, Z1_CommandVelocity decimal, Z1_CommandAcceleration decimal, Z1_CurrentFeedback decimal, Z1_DCBusVoltage decimal, Z1_OutputCurrent decimal, Z1_OutputVoltage decimal, S1_ActualPosit